In [8]:
import pandas as pd

game_metadata = pd.read_csv('./out/game_metadata.csv')
game_player_stats = pd.read_csv('./out/game_player_stats.csv')

First of we want a elo rating of each team before each major tournament for that we first sort all matches chronologically

In [9]:
#Convert date to datetime
game_metadata['date'] = pd.to_datetime(game_metadata['date'])
print(game_metadata.dtypes)

gameid                object
date          datetime64[ns]
league                object
playoffs               int64
patch                float64
gamelength             int64
dtype: object


In [10]:
#Actually sort the matches by date
game_metadata = game_metadata.sort_values(by='date').reset_index(drop=True)

Note: Import the team_name_map and player_name_map for now we can just use the team through id but would be useful when modelling/graphing it to match it to the teams

Now we need to calculate the elo of each team and save a snapshot of said elo before each tournament and each patch, the idea is as follows: 
1. Initialize the elo for each team with 1500
2. Create a list of major tournament start date (We will be looking at Worlds, MSI and First Stand for now)
3. Iterate over the matches: 
    4. Check whether the match is at a major tournament
    5. If yes
        6. Take a 'snapshot' of the whole elo ratings before said match
        7. Save it in a map where the key is the date of the first match of that tournament e.g. the date of the current match
    8. Update the elo ratings of the two teams using the result of the match
9. END

After we have a map of Elo rating lists for each team before each tournament 

In [11]:
#First we create a dataframe containing the elo of each 
all_teams = game_player_stats['teamid'].unique()
print(len(all_teams))

#Creates a map of elo ratings where we start at 1500 for each team
elo_ratings = {teamid: 1500 for teamid in all_teams}

1849


In [13]:
#Now we need to implement a function that calculates the Elo for each team after a match
#For now we use the elo algorithm see original paper also cite it
#Other canditates would be glicko2 or trueskill

#This function returns the updated ratings for each player in our case teams
#The result_A is modelled out of perspective of A thus 0 means A lost and B won and 1 means A won and B lost
#Using original formulas for expected outcome and update formula
def update_elo(rating_A, rating_B, result_A, k=32):
    #Calculate win probability for each team, e.g. the expected outcome
    expected_outcome_A = 1 / ( 1+10 **((rating_B-rating_A) / 400) )
    expected_outcome_B = 1- expected_outcome_A

    #Update the elo ratings
    new_rating_A = rating_A + k*(result_A - expected_outcome_A)
    new_rating_B = rating_B + k*((1 - result_A) - expected_outcome_B)

    return new_rating_A, new_rating_B

In [12]:
#Essentially we dont need to create a list for each major tournament since we can for each game look at
#which league that game was played and if it is either 'MSI', 'WLDs' and 'FST' we know we arrived at a major tournament
#For now a lot of unneccessary teams since we only look at tier 1 events but have tier 2 leagues included but doesnt matter

tournament_list = ['MSI', 'WLDs', 'FST']

gameid                object
date          datetime64[ns]
league                object
playoffs               int64
patch                float64
gamelength             int64
dtype: object


In [53]:
#Create a map that saves the elo snapshot
elo_snapshot = {}   #Key is the date of the first match of that tournament e.g the first row that has said tournament

#Now we iterate over the matches
for idx, row in game_metadata.iterrows():
    current_league = row['league']

    #Copy the a snapshot elo rating if current match is of a major tournament
    if current_league in tournament_list:
        elo_snapshot[row['date']] = elo_ratings.copy()
    
    #Now update the elo of each team
    #First get both teams
    game_entries = game_player_stats[game_player_stats['gameid'] == row['gameid']]
    assert len(game_entries) == 10
    teams_in_game = game_entries['teamid'].unique()
    team_A, team_B = teams_in_game

    #Now fetch the result 
    results_A = game_entries[game_entries['teamid'] == team_A]
    result_A = results_A['result'].unique()[0]

    #Finally actually update the ratings
    new_rating_A, new_rating_B = update_elo(elo_ratings[team_A], elo_ratings[team_B], result_A)
    elo_ratings[team_A] = new_rating_A
    elo_ratings[team_B] = new_rating_B


In [7]:
#Save the data 
import pickle 

#with open('./out/elo_ratings.pkl', 'wb') as f:
    #pickle.dump(elo_ratings, f)

#with open('./out/elo_snapshot.pkl', 'wb') as f:
    #pickle.dump(elo_snapshot, f)

NameError: name 'elo_snapshot' is not defined

In [25]:
#Load the data using pickle
with open('./out/elo_ratings.pkl', 'rb') as f:
    elo_ratings = pickle.load(f)
with open('./out/elo_snapshot.pkl', 'rb') as f:
    elo_snapshot = pickle.load(f)

In [26]:
#Look at highest ranked team
best_team = max(elo_ratings, key=elo_ratings.get)
print(f"Best team: {best_team} with rating {elo_ratings[best_team]}")

Best team: oe:team:50f58982d91a36557ec8aec52ab014f with rating 2016.2674582956067


In [27]:
#Search for the first Worlds tournament and see whether the elo snapshot was saved properly
tournament = ['WLDs']
tournament_matches = game_metadata[game_metadata['league'].isin(tournament)]

tournament_matches = tournament_matches.sort_values(by='date')
first_tournament_game = tournament_matches.iloc[0]
print(first_tournament_game)
elo_snapshot_test = elo_snapshot[first_tournament_game['date']]
print(elo_snapshot_test)
best_team = max(elo_snapshot_test, key=elo_snapshot_test.get)
print(f"Best team: {best_team} with rating {elo_ratings[best_team]}")

gameid           TRLH3/1000430019
date          2014-09-18 08:59:56
league                       WLDs
playoffs                        0
patch                        4.14
gamelength                   2637
Name: 696, dtype: object
{'oe:team:78f183fa5a7d8ecb22b9ad272c3abd7': 1628.1252386974027, 'oe:team:3e18c32708da93cf6bdbd1d20b76731': 1462.9154445541421, 'oe:team:6d92c120d5550d72128830b0a4df0f8': 1479.1354864285884, 'oe:team:05bd29b524f0851d42e7902d6af59bf': 1685.5379648669714, 'oe:team:75760dca486d4682426196b76bd1a54': 1605.4841338609797, 'oe:team:d946eb399b263986b81fad7203186fa': 1392.7348341529294, 'oe:team:3de6f8727f5d53282e39da4f859931d': 1550.8489274787737, 'oe:team:907351811cc0343ae5746cd2c75549d': 1481.8657120121038, 'oe:team:fc8e90107dabb9a35c490b0d86adea0': 1694.0805987801305, 'oe:team:a22510a4756fa424cb0771b13261139': 1673.4749690369672, 'oe:team:f7f7324c151aa6ed770b244ce08e027': 1553.0713519324452, 'oe:team:06ff5564847128343223be7514c8193': 1338.7803282621865, 'oe:team:930d1